<a href="https://colab.research.google.com/github/Jayaaa508/sentiment-analysis/blob/main/streamlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
import pickle as pkl
file = open('m.pkl','rb')
nb = pkl.load(file)
file.close()



In [37]:

!pip install spacy
!python -m spacy download en_core_web_lg
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 1.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 38.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [38]:
import spacy

nlp = spacy.load("en_core_web_lg")

In [27]:
!pip install streamlit

In [61]:
%%writefile stream.py
import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from sklearn.naive_bayes import MultinomialNB
import pickle as pkl
from pickle import load
import spacy
nlp = spacy.load("en_core_web_lg")
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

# for preprocessing
def preprocess(text):
    doc = nlp(text)
    filtered_tokens = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
    return " ".join(filtered_tokens)

# vectorise
def vectorize(text):
    return nlp(text).vector.reshape(1, -1)

# for finding the bias
def get_bias(text):
    txt = preprocess(text)
    embeddings = vectorize(txt)
    prediction = __model.predict(embeddings)[0]

    if prediction == -1:
        return "Negative"
    elif prediction == 1:
        return "Positive"
    else:
        return "Neutral"

# for loading the model
def load_saved_model():
    global __model
    try:
        with open("m.pkl", "rb") as f:
            __model = load(f)
        st.success("Model loaded successfully.")
    except FileNotFoundError:
        st.error("Model file not found. Please check the file path.")
    except Exception as e:
        st.error(f"Error loading model: {e}")

def main():
    load_saved_model()
  # Streamlit app setup
    st.title("Sentiment Analysis Report")
    st.sidebar.title("Navigation")
    st.sidebar.markdown("Select a section to view")

    # Sidebar options
    options = st.sidebar.selectbox("Sections", ["Introduction", "Data Overview", "Exploratory Data Analysis", "Sentiment Analysis" ,"Vizualization", "Conclusion"])

    # Introduction section
    if options == "Introduction":
        st.header("Introduction")
        st.markdown("""
        This report provides an analysis of social media sentiment data. The analysis includes data processing,
        exploratory data analysis (EDA), sentiment analysis using predictive models, and visualization of the results.
        """)

    # Data Overview section
    elif options == "Data Overview":
        st.header("Data Overview")
        df = pd.read_csv("raw.csv", encoding ="latin-1")
        st.subheader("Dataset")
        st.write(df.head())
        st.markdown(f"Number of records: {df.shape[0]}")
        st.markdown(f"Number of features: {df.shape[1]}")

    # Exploratory Data Analysis section
    elif options == "Exploratory Data Analysis":
        st.header("Exploratory Data Analysis (EDA)")
        st.subheader("Distribution of Sentiments")
        df = pd.read_csv("preprossed.csv", encoding ="latin-1")
        sentiment_counts = df['sentiment'].value_counts()
        st.bar_chart(sentiment_counts)
        st.markdown(f"Number of records: {df.shape[0]}")
        st.markdown(f"Number of features: {df.shape[1]}")

    #Sentiment Analysis
    elif options == "Sentiment Analysis":
        st.header("Sentiment Analysis")
        st.subheader("Predict Sentiment")
        text = st.text_area('Enter your comment here:', height=200, placeholder="Type your text here...")
        if st.button('Predict'):
            if __model is not None:
                prediction = get_bias(text)
            if prediction == "Positive":
                      st.success(f'Predicted Sentiment: {prediction}')
            elif prediction == "Negative":
                        st.error(f'Predicted Sentiment: {prediction}')
            else:
                            st.info(f'Predicted Sentiment: {prediction}')
        else:
            st.error("Model not loaded. Please check the model file.")

    # vizualization of data set
    elif options == "Vizualization":
        st.header("Data Vizualization ")
        data = pd.read_csv("preprossed.csv")
        st.subheader("Histogram ")
        fig, ax = plt.subplots()
        sns.histplot(data=data, x='sentiment', ax=ax)
        ax.set_title('Distribution of Sentiment Scores')
        ax.set_xlabel('Sentiment Score')
        ax.set_ylabel('Frequency')
        st.pyplot(fig)


        st.subheader("Word cloud")
        df = pd.read_csv("preprossed.csv", encoding ="latin-1")

        comment_words = ''
        stopwords = set(STOPWORDS)

        # iterate through the csv file
        for val in df.text:

            # typecaste each val to string
            val = str(val)

            # split the value
            tokens = val.split()

            # Converts each token into lowercase
            for i in range(len(tokens)):
                tokens[i] = tokens[i].lower()

            comment_words += " ".join(tokens)+" "

        wordcloud = WordCloud(width = 800, height = 800,
                        background_color ='white',
                        stopwords = stopwords,
                        min_font_size = 10).generate(comment_words)

        # plot the WordCloud image
        plt.figure(figsize = (8, 8), facecolor = None)
        plt.imshow(wordcloud)
        plt.axis("off")
        plt.tight_layout(pad = 0)
        st.pyplot()


        # Display the word cloud

        showPyplotGlobalUse = false
    # Conclusion section
    elif options == "Conclusion":
        st.header("Conclusion")
        st.markdown("""
        The analysis provides insights into the sentiment trends and distribution across various categories.
        Future work can include deeper sentiment analysis using advanced NLP models and expanding the dataset
        to include more sources.
        """)
        st.write('Made by Jaya Agarwal')

    #if __name__ == '__main__':

main()





Overwriting stream.py


In [59]:
! wget -q -O - ipv4.icanhazip.com

34.125.155.108


In [62]:
! streamlit run stream.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.2:8501
  External URL: http://34.125.155.108:8501

npx: installed 22 in 2.913s
your url is: https://true-wolves-feel.loca.lt
2024-07-05 21:27:15.136 Uncaught app exception
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/streamlit/runtime/scriptrunner/script_runner.py", line 589, in _run_script
    exec(code, module.__dict__)
  File "/content/stream.py", line 166, in <module>
    main()
  File "/content/stream.py", line 153, in main
    showPyplotGlobalUse = false
NameError: name 'false' is not defined
  Stopping...
^C
